In [9]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd

In [10]:
def get_jobs(keyword, num_jobs, verbose):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path="C:/Users/mundl/Documents/ds_salary_project/chromedriver", options=options)
    driver.set_window_size(1120, 1000)

    url = "https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=&locT=&locId=&jobType=&context=Jobs&sc.keyword=%22data+scientist%22&dropdown=0"
    #url = 'https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=data+science&locT=&locId=&jobType=&context=Jobs&sc.keyword=data+science&dropdown=0'
    driver.get(url)
    jobs = []

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(4)

        #Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element_by_class_name("selected").click()
        except ElementClickInterceptedException:
            pass

        time.sleep(.1)

        try:
            driver.find_element_by_css_selector('[alt="Close"]').click()  #clicking to the X.
            print(' x out worked')
        except NoSuchElementException:
            print(' x out failed')
            pass

        
        #Going through each job in this page
        job_buttons = driver.find_elements_by_class_name("jl")  #jl for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:  

            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break

            job_button.click()  #You might 
            time.sleep(1)
            collected_successfully = False
            
            while not collected_successfully:
                try:
                    company_name = driver.find_element_by_xpath('.//div[@class="employerName"]').text
                    location = driver.find_element_by_xpath('.//div[@class="location"]').text
                    job_title = driver.find_element_by_xpath('.//div[contains(@class, "title")]').text
                    job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(5)

            try:
                salary_estimate = driver.find_element_by_xpath('.//span[@class="gray salary"]').text
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
            
            try:
                rating = driver.find_element_by_xpath('.//span[@class="rating"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                driver.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="overview"]').click()

                try:
                    #<div class="infoEntity">
                    #    <label>Headquarters</label>
                    #    <span class="value">San Francisco, CA</span>
                    #</div>
                    headquarters = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                except NoSuchElementException:
                    headquarters = -1

                try:
                    size = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1

                try:
                    competitors = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
                except NoSuchElementException:
                    competitors = -1

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                competitors = -1

                
            if verbose:
                print("Headquarters: {}".format(headquarters))
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("Competitors: {}".format(competitors))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Headquarters" : headquarters,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue,
            "Competitors" : competitors})
            #add job to jobs

        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//li[@class="next"]//a').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [11]:
df = get_jobs("data scientist", 15, False)
df

 x out worked
Progress: 0/15
Progress: 1/15
Progress: 2/15
Progress: 3/15
Progress: 4/15
Progress: 5/15
Progress: 6/15
Progress: 7/15
Progress: 8/15
Progress: 9/15
Progress: 10/15
Progress: 11/15
Progress: 12/15
Progress: 13/15
Progress: 14/15
Progress: 15/15


,Company Name,Competitors,Founded,Headquarters,Industry,Job Description,Job Title,Location,Rating,Revenue,Salary Estimate,Sector,Size,Type of ownership
0,Pacific Northwest National Laboratory\n3.8,"Oak Ridge National Laboratory, National Renewa...",1965,"Richland, WA",Energy,*Organization and Job ID**\nJob ID: 310918\n\n...,"Data Scientist - Statistics, Early Career","Seattle, WA",3.8,$500 million to $1 billion (USD),$68K-$111K (Glassdoor est.),"Oil, Gas, Energy & Utilities",1001 to 5000 employees,Government
1,DentaQuest\n3.2,"MCNA Dental Plans, United Concordia, Delta Den...",2001,"Boston, MA",Insurance Carriers,This position is responsible for collaborating...,Senior Healthcare Analyst/Data Scientist,"Milwaukee, WI",3.2,$2 to $5 billion (USD),$68K-$111K (Glassdoor est.),Insurance,1001 to 5000 employees,Company - Private
2,Evernote\n4.0,-1,2008,"Redwood City, CA",Computer Hardware & Software,About the team:\n\n\nThe Evernote Analytics te...,"Data Scientist, Analytics",United States,4.0,$50 to $100 million (USD),$68K-$111K (Glassdoor est.),Information Technology,201 to 500 employees,Company - Private
3,Juniper Networks\n3.8,-1,1996,"Sunnyvale, CA",Telecommunications Services,"Data Scientist/Machine Learning\n\nMist, a Jun...",Data Scientist/Machine Learning,"Cupertino, CA",3.8,$2 to $5 billion (USD),$68K-$111K (Glassdoor est.),Telecommunications,5001 to 10000 employees,Company - Public
4,Zynga\n4.1,-1,2007,"San Francisco, CA",Video Games,Zynga is a leading developer of the world’s mo...,Data Scientist (Engineering),"Austin, TX",4.1,$500 million to $1 billion (USD),$68K-$111K (Glassdoor est.),Media,1001 to 5000 employees,Company - Public
5,VA Boston Healthcare System\n3.6,-1,-1,"Boston, MA",Health Care Services & Hospitals,"VA Boston Healthcare System\nBoston, Massachus...",Data Scientist,"Boston, MA",3.6,Unknown / Non-Applicable,$68K-$111K (Glassdoor est.),Health Care,201 to 500 employees,Hospital
6,Netskope\n4.2,"Skyhigh Networks, Zscaler, NortonLifeLock",2012,"Santa Clara, CA",Enterprise Software & Network Solutions,"About Netskope\n\nToday, there's more data and...",Data Scientist,"San Francisco, CA",4.2,Unknown / Non-Applicable,$68K-$111K (Glassdoor est.),Information Technology,501 to 1000 employees,Company - Private
7,First Orion\n4.8,-1,2008,"North Little Rock, AR",Enterprise Software & Network Solutions,First Orion is actively seeking a talented Dat...,Data Scientist Intern,"North Little Rock, AR",4.8,Unknown / Non-Applicable,$68K-$111K (Glassdoor est.),Information Technology,201 to 500 employees,Company - Private
8,MMIT\n3.9,-1,1994,"Yardley, PA",Biotech & Pharmaceuticals,Company Description\n\nMMIT is growing from th...,Data Scientist,"Yardley, PA",3.9,Unknown / Non-Applicable,$68K-$111K (Glassdoor est.),Biotech & Pharmaceuticals,201 to 500 employees,Company - Private
9,NAVEX Global\n3.2,-1,2012,"Lake Oswego, OR",Computer Hardware & Software,It's fun to work in a company where people tru...,Data Scientist,"Lake Oswego, OR",3.2,$100 to $500 million (USD),$68K-$111K (Glassdoor est.),Information Technology,1001 to 5000 employees,Company - Private
